In [1]:
import numpy as np
import os
import multiprocessing
import argparse
import tensorflow as tf
import tensorflow.contrib.slim as slim

from tensorpack import InputDesc, ModelDesc, logger, QueueInput
from tensorpack.models import *
from tensorpack.callbacks import *
from tensorpack.dataflow import BatchData,PrefetchDataZMQ
from tensorpack.tfutils import SaverRestore

from tensorpack.train import TrainConfig, SyncMultiGPUTrainer
import tensorpack.tfutils.symbolic_functions as symbf
from tensorpack.tfutils.summary import add_moving_summary
from tensorpack.tfutils import argscope, get_model_loader
from tensorpack.utils.gpu import get_nr_gpu

IMAGE_BATCH_SIZE = 33

import sys
#sys.path.append('../ArodProcessing')
#import arod_dataflow_from_txt
#import arod_provider

IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
BATCH_SIZE = 32

def bn_with_gamma(x):
    return slim.layers.batch_norm(inputs=x, scale=True)

def resnet_shortcut(shortcut, features, stride):

    in_channel = shortcut.get_shape().as_list()[3]

    if in_channel != features:  # features == input channels???
        shortcut = slim.layers.conv2d(inputs=shortcut,
                                      num_outputs=features,
                                      kernel_size=1,
                                      stride=stride,
                                      padding='same',
                                      normalizer_fn=slim.layers.batch_norm,
                                      activation_fn=None,
                                      scope='convshortcut')
    return shortcut


def resnet_block(x, features, stride):

    shortcut = resnet_shortcut(x, features * 4, stride)
    x = slim.layers.conv2d(inputs=x,
                           num_outputs=features,
                           kernel_size=1,
                           stride=1,
                           padding='same',
                           normalizer_fn=slim.layers.batch_norm,
                           scope='conv1')

    x = slim.layers.conv2d(inputs=x,
                           num_outputs=features,
                           kernel_size=3,
                           stride=stride,
                           padding='same',
                           normalizer_fn=slim.layers.batch_norm,
                           scope='conv2')

    x = slim.layers.conv2d(inputs=x,
                           num_outputs=features * 4,
                           kernel_size=1,
                           stride=1,
                           padding='same',
                           normalizer_fn=bn_with_gamma,
                           activation_fn=None,
                           scope='conv3')

    return x + shortcut


def resnet_group(x, name, features, stride, blocks):
    with tf.variable_scope(name):
        for i in range(0, blocks):
            with tf.variable_scope('block{}'.format(i)):
                x = resnet_block(x, features, stride if i == 0 else 1)
                x = tf.nn.relu(x)
    return x

def _get_inputs():
    return [(tf.float32, (BATCH_SIZE, IMAGE_WIDTH, IMAGE_HEIGHT, 3), 'input_a'),
            InputDesc(tf.float32, (BATCH_SIZE,), 'score_a'),
            InputDesc(tf.float32, (BATCH_SIZE, IMAGE_WIDTH, IMAGE_HEIGHT, 3), 'input_p'),
            InputDesc(tf.float32, (BATCH_SIZE,), 'score_p'),
            InputDesc(tf.float32, (BATCH_SIZE, IMAGE_WIDTH, IMAGE_HEIGHT, 3), 'input_n'),
            InputDesc(tf.float32, (BATCH_SIZE,), 'score_n')]




def embed(nfeatures=1000):
#     if isinstance(x, list):
#         x = tf.concat(x, 0)
    # 1st Layer
    x = tf.placeholder(tf.float32, shape=(IMAGE_BATCH_SIZE,224,224,3), name='image_input')
    x = slim.layers.conv2d(x, kernel_size=7, stride=2, num_outputs=64,
                           normalizer_fn=slim.layers.batch_norm, scope='conv0')
    x = slim.layers.max_pool2d(x, kernel_size=3, stride=2, scope='pool0')
    # Residual Blocks
    x = resnet_group(x=x, features=64, stride=1, name='group0', blocks=3)
    x = resnet_group(x=x, features=128, stride=2, name='group1', blocks=4)
    x = resnet_group(x=x, features=256, stride=2, name='group2', blocks=6)
    x = resnet_group(x=x, features=512, stride=2, name='group3', blocks=3)
    x = tf.reduce_mean(x, [1, 2])
    # 8th Layer: FC and return unscaled activations
    embedding = slim.layers.fully_connected(x, nfeatures, activation_fn=None, scope='embedding')
    return embedding

In [2]:
net = embed()

In [10]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter('logs', sess.graph)
    print sess.run(net)
    writer.close()

InvalidArgumentError: You must feed a value for placeholder tensor 'image_input' with dtype float and shape [33,224,224,3]
	 [[Node: image_input = Placeholder[dtype=DT_FLOAT, shape=[33,224,224,3], _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: embedding/BiasAdd/_1 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_719_embedding/BiasAdd", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op u'image_input', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/capptu/envs/TensorFlow/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/capptu/envs

In [4]:
images28 = np.random.random((IMAGE_BATCH_SIZE,224,224,3))
labels = np.random.random((IMAGE_BATCH_SIZE,1024))





In [5]:
from tensorflow import array_ops
a = array_ops.placeholder(dtypes.float32, shape=[])
b = array_ops.placeholder(dtypes.float32, shape=[])
c = array_ops.placeholder(dtypes.float32, shape=[])
r1 = math_ops.add(a, b)
r2 = math_ops.multiply(r1, c)

h = sess.partial_run_setup([r1, r2], [a, b, c])
res = sess.partial_run(h, r1, feed_dict={a: 1, b: 2})
res = sess.partial_run(h, r2, feed_dict={c: res})


ImportError: cannot import name array_ops

In [6]:
x = tf.constant([[37.0, -23.0], [1.0, 4.0]])
w = tf.Variable(tf.random_uniform([2, 2]))
y = tf.matmul(x, w)
# ...
loss = ...
train_op = tf.train.AdagradOptimizer(0.01).minimize(loss)

with tf.Session() as sess:
  # `sess.graph` provides access to the graph used in a <a href="./../api_docs/python/tf/Session"><code>tf.Session</code></a>.
  writer = tf.summary.FileWriter("/tmp/log/...", sess.graph)

  # Perform your computation...
  for i in range(1000):
    sess.run(train_op)
    # ...

  writer.close()


SyntaxError: invalid syntax (<ipython-input-6-b6d47d3964ce>, line 5)